In [ ]:
# %cd to your file  

In [ ]:
# Import functions 
from grab_data_from_matlab import *
from grab_look_mgs_correct_index import *
from change_memang_to_integer import * 
from preprocess_neurons import preprocess_neurons_by_creating_new_index_list
from grab_corresponding_memang import *
from grab_corresponding_data import * 
from change_int_pos_to_float_memang import *
from circ_stat_functions import *

In [ ]:
# Import packages 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure 
import seaborn as sns
import scipy.io
import random
import math
from scipy import special
from sympy import *
from scipy.io import savemat
from scipy.optimize import curve_fit

# Logistic Regression Classifier setup
from numpy import mean 
from numpy import std 
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression 
import scipy.stats as stats

In [ ]:
## Building Input
total_neurons, task_and_result, memang = grab_data("aq_20210921_cells.mat")
look_mgs_correct_index, stimulus_position = grab_look_mgs_correct_index(task_and_result, memang)
corresponding_neurons = grab_corresponding_data(total_neurons, look_mgs_correct_index)
float_stim_pos, pre_int_stimulus_position = change_memang_to_int(stimulus_position)
preprocessed_index_list = preprocess_neurons_by_creating_new_index_list(pre_int_stimulus_position)

# Final corresponding neurons and int stimulus position (memang)
new_corresponding_neurons = grab_new_corresponding_data(total_neurons, corresponding_neurons, preprocessed_index_list)
post_int_stimulus_position = grab_corresponding_memang(pre_int_stimulus_position, preprocessed_index_list)

In [ ]:
np.save("new_corresponding_neurons", new_corresponding_neurons)
np.save("integer_stimulus_position", post_int_stimulus_position)

In [ ]:
# Change the int stimulus position to float 
stim_pos_to_float = change_int_pos_to_float_memang(post_int_stimulus_position, float_stim_pos)

In [ ]:
## Analysis 1
# Z-score relative to neuron (axis=0) & leave one out cross validation & multi-logistic regression decoder 

# Necessary set up 
loo = LeaveOneOut()
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
confidence_matrix = np.zeros((total_neurons.shape[2],len(preprocessed_index_list), 8)) #(200,376,8)
confidence_matrix[:] = np.nan
post_int_stimulus_position = np.array(post_int_stimulus_position)

## Multi-classification using the multinomial logistic regression
## This does not provide score, but provides confidence matrix of the classifer. (200:tc, 376:trial, 8: stimulus position)
for tc_num in range(total_neurons.shape[2]):
    print(tc_num)
    placeholder = new_corresponding_neurons[:,:,tc_num]
    placeholder = placeholder.T # trials x neurons 
    neuron_zscore = stats.zscore(placeholder, axis=0, nan_policy='propagate')
    norm_relative_to_neurons = neuron_zscore[:,~np.isnan(neuron_zscore).any(axis=0)]
    
    X = norm_relative_to_neurons
    y = post_int_stimulus_position
    
    y_predict_prob = cross_val_predict(model, X, y, cv=loo, method='predict_proba')
    confidence_matrix[tc_num,:,:] = y_predict_prob

In [ ]:
# Saving appropriate matrices and stimulus position vectors 
np.save("new_conf_matrix", confidence_matrix)
np.save("stim pos float vector", stim_pos_to_float)
np.save("stim pos int vector", post_int_stimulus_position)

# Save in matlab data 
savemat('memang_float_vector.mat', {'memang_float_vector': stim_pos_to_float})
savemat('memang_int_vector.mat', {'memang_int_vector': post_int_stimulus_position})

In [ ]:
# If I need to load confidence matrix and stim_pos_to_float
confidence_matrix = np.load("new_conf_matrix.npy")
stim_pos_to_float = np.load("stim pos float vector.npy")

In [ ]:
# Input to curve fitting 
unique_stim_position_float = np.unique(stim_pos_to_float)

In [ ]:
# Analysis 2
# Von mises curve fitting 
# Define some parameters
binsize_generative = 2*np.pi/8
binsize_fit        = 2*np.pi/100

# generate x values
theta_generative = np.arange(-np.pi, np.pi - binsize_generative, binsize_generative)
theta_fit = np.arange(-np.pi, np.pi - binsize_fit, binsize_fit)
theta_generative = np.append(theta_generative, 2.35619449)
theta_fit = np.append(theta_fit, 3.0787608100000003)

# Data obtained using the von mises function originally from matlab (resultant, mu, sd, kappa)
resultant_magnitude_matrix = np.zeros((confidence_matrix.shape[1],confidence_matrix.shape[0]))
mu_fit = np.zeros((confidence_matrix.shape[1],confidence_matrix.shape[0]))  #376*200 (number of trials x number of tc)
sd_fit = np.zeros((confidence_matrix.shape[1],confidence_matrix.shape[0]))
kappa_fit = np.zeros((confidence_matrix.shape[1],confidence_matrix.shape[0]))

# Curve fitted mu, kappa, peak values
curve_fitted_mu = np.zeros((confidence_matrix.shape[1],confidence_matrix.shape[0]))
curve_fitted_k = np.zeros((confidence_matrix.shape[1],confidence_matrix.shape[0]))
curve_fitted_peak = np.zeros((confidence_matrix.shape[1],confidence_matrix.shape[0]))
curve_fitted_peak_confidence = np.zeros((confidence_matrix.shape[1],confidence_matrix.shape[0]))
curve_fitted_alpha = np.zeros((confidence_matrix.shape[1],confidence_matrix.shape[0]))

for trial_num in range(confidence_matrix.shape[1]):
    for tc_num in range(confidence_matrix.shape[0]):
        
        print("trial_num", trial_num)
        print("tc_num", tc_num)
        
        # Create resultant matrix (using von mises)
        result = np.abs(cresultant(theta_generative, confidence_matrix[tc_num, trial_num,:]))
        resultant_magnitude_matrix[trial_num, tc_num] = result

        # Create mu fit matrix (using von mises)
        M = cmean(theta_generative, confidence_matrix[tc_num, trial_num,:])
        mu_fit[trial_num, tc_num] = M 
        
        # Create sd_fit matrix (using von mises)
        S = cstd(theta_generative, confidence_matrix[tc_num, trial_num,:])
        sd_fit[trial_num, tc_num] = S
        
        # Create kappa fit matrix (using von mises)
        K = sd2k(S) 
        kappa_fit[trial_num, tc_num] = K 
        
        x = unique_stim_position_float
        y = confidence_matrix[tc_num, trial_num,:]
        
        # Von mises curve fitting 

        popt, _ = curve_fit(objective, x, y, p0 = [20, 50, 1], bounds=([0, 0, -np.pi], [50, 50, np.pi]), maxfev=1000000)
        alpha, k, mu = popt
        pdf_fit = objective(theta_fit, alpha, k, mu)
        
        curve_fitted_alpha[trial_num, tc_num] = alpha
        curve_fitted_mu[trial_num, tc_num] = mu
        curve_fitted_k[trial_num, tc_num] = k
        
        curve_peakConfidence = objective(mu, alpha, k, mu) 
#         # curve_peakConfidence = peakConfidence(mu, alpha, k, mu) - same line
        curve_fitted_peak_confidence[trial_num, tc_num] = curve_peakConfidence
        
        peak = np.max(pdf_fit)
        curve_fitted_peak[trial_num, tc_num] = peak

In [ ]:
# # Save all the generated matrices 
# np.save("resultant_magnitude", resultant_magnitude_matrix)
# np.save("mu_fit", mu_fit)
# np.save("sd_fit", sd_fit)
# np.save("kappa_fit", kappa_fit)
# np.save("curve_fitted_mu", curve_fitted_mu)
# np.save("curve_fitted_k", curve_fitted_k)
# np.save("curved_fitted_peak", curved_fitted_peak)
np.save("curve_fitted_alpha", curve_fitted_alpha)

# # Save resultant and curve fitted maximum peak value to matlab data format 
# savemat('resultant_magnitude_matrix.mat', {'resultant_magnitude_matrix': resultant_magnitude_matrix})
# savemat('curve_fitted_peak.mat', {'curve_fitted_peak': curved_fitted_peak})

In [ ]:
# Plotting the difference between mu_fit and memang - 
# Take an absolute value of the subtraction between mu and original float stimulus pos 

subtracted_matrix = np.zeros((376,200))
for trial_num in range(mu_fit.shape[0]):
    subtraction = np.subtract(mu_fit[trial_num,:], stim_pos_to_float[trial_num])
    subtracted_matrix[trial_num,:] = subtraction
processed_subtracted_matrix = np.abs(subtracted_matrix)
mean_processed_subtracted_matrix = np.mean(processed_subtracted_matrix, axis=0)

x=np.linspace(-500,1500, num=len(mean_processed_subtracted_matrix))
plt.plot(x,mean_processed_subtracted_matrix)
plt.xlabel('Time from Stimulus onset (ms)')
plt.ylabel('Absolute diff mean' )
plt.title('Absolute difference between mu and memang - mean across trials')
plt.show()

In [ ]:
# Plot SD Fit, average across trials 
mean_sd_fit = np.mean(sd_fit, axis=0)
x=np.linspace(-500,1500, num=len(mean_sd_fit))
plt.plot(x, mean_sd_fit)
plt.xlabel('Time from Stimulus onset (ms)')
plt.ylabel('Mean SD Fit')
plt.title('SD Fit (mean across trials)')

In [ ]:
# Plot resultant, average across trials
mean_resultant_magnitude_vec = np.mean(resultant_magnitude_matrix, axis=0)
x=np.linspace(-500,1500, num=len(mean_resultant_magnitude_vec))
plt.plot(x,mean_resultant_magnitude_vec)
plt.xlabel('Time from Stimulus onset (ms)')
plt.ylabel('Absolute Mean value of resultant')
plt.title('Resultant')

In [ ]:
# Mean across the trials, peak of the von mises curve fitting plot 
# peak_plot_matrix = np.load("peak_plot_matrix.npy")

mean_peak = np.mean(curved_fitted_peak, axis=0)
x=np.linspace(-500,1500, num=len(mean_peak))
plt.plot(x,mean_peak)
plt.xlabel('Time from Stimulus onset (ms)')
plt.ylabel('Mean Maximum Peak Confidence Fit')
plt.title('Mean Maximum Peak Confidence Fit (np.max)')

In [ ]:
curved_fitted_mean_peak_confidence = np.mean(curved_fitted_peak_confidence, axis=0)
x=np.linspace(-500,1500, num=len(curved_fitted_mean_peak_confidence))
plt.plot(x,mean_peak)
plt.xlabel('Time from Stimulus onset (ms)')
plt.ylabel('Mean Peak Confidence Fit')
plt.title('Mean Peak Confidence Fit')

In [ ]:
subtracted_matrix_2 = np.zeros((376,200))
for trial_num in range(curve_fitted_mu.shape[0]):
    subtraction = np.subtract(curve_fitted_mu[trial_num,:], stim_pos_to_float[trial_num])
    subtracted_matrix_2[trial_num,:] = subtraction
processed_subtracted_matrix_2 = np.abs(subtracted_matrix_2)
mean_processed_subtracted_matrix_2 = np.mean(processed_subtracted_matrix_2, axis=0)

x=np.linspace(-500,1500, num=len(mean_processed_subtracted_matrix_2))
plt.plot(x,mean_processed_subtracted_matrix_2)
plt.xlabel('Time from Stimulus onset (ms)')
plt.ylabel('Absolute diff mean' )
plt.title('Absolute difference between curve fitted mu and memang - mean across trials')
plt.show()

In [ ]:
# Curve fitted k
mean_curve_fitted_k = np.mean(curve_fitted_k, axis=0)
x=np.linspace(-500,1500, num=len(mean_curve_fitted_k))
plt.plot(x,mean_curve_fitted_k)
plt.xlabel('Time from Stimulus onset (ms)')
plt.ylabel('Curve fitted K')
plt.title('Mean Curve fitted K')

In [ ]:
# Kappa
mean_kappa_fit = np.mean(kappa_fit, axis=0)
x=np.linspace(-500,1500, num=len(mean_kappa_fit))
plt.plot(x,mean_kappa_fit)
plt.xlabel('Time from Stimulus onset (ms)')
plt.ylabel('Kappa Fit')
plt.title('Mean Kappa Fit ')